In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import imutils

In [ ]:
img1 = cv2.imread("data/ljubljana_1.png")
img2 = cv2.imread("data/ljubljana_2.png")
img3 = cv2.imread("data/ljubljana_3.png")
img4 = cv2.imread("data/ljubljana_4.png")
images = [img1, img2, img3, img4]

In [ ]:
def prikazi_sliko(img):
    cv2.imshow("slika", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
def skaliraj_sliko(img, faktor):
    sirina = int(img.shape[1] / faktor)
    visina = int(img.shape[0] / faktor)
    dim = (sirina, visina)
    return cv2.resize(img, dim,  interpolation = cv2.INTER_AREA)

In [ ]:
slike_za_obdelavo = cv2.createStitcher()

In [ ]:
(status, slika_spojena) = slike_za_obdelavo.stitch(images)

In [ ]:
if status == 0:
    print("Spenjanje slik je uspelo")
    print("Dimenzija spojene slike:    ", slika_spojena.shape)

In [ ]:
slika_spojena = skaliraj_sliko(slika_spojena, 2)

In [ ]:
slika_spojena = cv2.copyMakeBorder(slika_spojena, 10, 10, 10, 10, cv2.BORDER_CONSTANT, (0, 0, 0))

In [ ]:
cv2.imshow("Rezultat", slika_spojena)
cv2.waitKey(0)

In [ ]:
slika_sivinska = cv2.cvtColor(slika_spojena, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(slika_sivinska, 0, 255, cv2.THRESH_BINARY)[1]

In [ ]:
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
obroba = imutils.grab_contours(cnts)
obroba = obroba[0]

In [ ]:
plt.plot(obroba[:,:, 0], obroba[:,:, 1], 'b')
# povezi prvo in zadnjo tocko
plt.plot([obroba[0,:,0], obroba[-1,:,0]], [obroba[0,:,1], obroba[-1,:,1]], 'b')
plt.title("Zunanji rob spojene slike")

In [ ]:
obroba = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# zajamemo ogljisca obrobe
obroba = imutils.grab_contours(obroba)

In [ ]:
# vrednosti v array
c = max(obroba, key=cv2.contourArea)

In [ ]:
# ustvarimo masko obmocja znotraj slike
mask = np.zeros(thresh.shape, dtype="uint8")
(x, y, w, h) = cv2.boundingRect(c)
# argumenti: zacetek, konec, barva, debelina
cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)

In [ ]:
# kopije maske
# prva za obmocje najmanjsega kvadrata slike
# druga je stevec za iteracijo zmanjsevanja kvadrata
minRect = mask.copy()
sub = mask.copy()

In [ ]:
# iteriramo dokler niso vse vrednosti vecje od nic
while cv2.countNonZero(sub) > 0:
    # zmanjsaj velikost maske nato odstej podrocja
    minRect = cv2.erode(minRect, None)
    sub = cv2.subtract(minRect, thresh)
# rezultat je minRect, maska z najmanjsim obmocjem slike

In [ ]:
# poiscemo obrobo pripravljenega najmanjsega kvadrata
obroba_prava = cv2.findContours(minRect.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
obroba_prava = imutils.grab_contours(obroba_prava)
# v array
c = max(obroba_prava, key=cv2.contourArea)
# narisemo kvadrat
(x, y, w, h) = cv2.boundingRect(c)

In [ ]:
# uporabimo vrednosti odmikov kvadrata za zmanjsanje obsega spojene slike
slika_spojena = slika_spojena[y:y + h, x:x + w]

In [ ]:
cv2.imwrite("matoda1_oSlika.png", slika_spojena) 

In [ ]:
# prikazi koncni rezultat
cv2.imshow("slika", slika_spojena)
while True:
    key = cv2.waitKey(0)
    if key in [27, ord('q'), ord('Q')]:
        cv2.destroyAllWindows()